In [2]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

# 1 Multiclass classification

## 1.1. Dataset

In [35]:
matdata = sio.loadmat("ex3data1.mat", struct_as_record = False)

global y
global X
global y_one
global lambda_

X = matdata['X'] 
y = matdata['y']

print("X shape:",X.shape)
print("y shape:",y.shape)

('X shape:', (5000, 400))
('y shape:', (5000, 1))


## 1.2 Visualizing the data

In [36]:
fig = plt.figure()
y_labels = np.zeros((25))
for i in range(1,26):
    idx = i*np.random.randint(1,150) # Random image index
    
    # Show image
    ax = fig.add_subplot(5,5,i)
    ax.matshow(X[idx,:].reshape(20,20), interpolation='nearest', cmap=plt.cm.gray)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    y_labels[i-1] = (y[idx][0]) # Add label

print("Labels:")
print(y_labels.reshape(5,5))

Labels:
[[ 10.  10.  10.  10.  10.]
 [ 10.   1.   1.   2.   1.]
 [  2.   2.   3.  10.   1.]
 [  1.   2.   1.   4.   5.]
 [  4.   4.   2.   5.   4.]]


## 1.3 Vectorizing logistic regression

### 1.3.1, 1.3.2, 1.3.3 Vectorizing cost function and gradient

In [37]:
def sigmoid(z):
    """Logistic regression func, returns probability"""
    return 1/(1+np.exp(-z))

def lrCostFunction(theta,X_,y_,lambda_arg):
    """Unregularized cost function"""
    J = 0
    m = y_.size
    grad = np.zeros((theta.size,1))
    
    # correct theta shape
    theta.shape = (theta.shape[0],1)
    
    h = sigmoid(X.dot(theta))
    h.shape = (h.size,1)
    
    J = (-1.0/m) * sum( y_.T.dot(np.log(h)) + (1-y_.T).dot(np.log(1-h)) )
    # Regularize the cost function - except theta[0]
    J += (lambda_arg/(2.0*m)) * sum(np.power(theta[1:],2))
    
    
    grad = (1.0/m) * sum((h-y_).T.dot(X_))
    grad.shape = (grad.shape[0],1) # correct shape
    # Regularize grad, except grad[0]
    grad[1:] += (lambda_arg/m) * theta[1:] 
    
    # Fminunc expects single value
    return J, grad.T.flatten()

def decorated_lrCostFunction(theta):
    """Cost function wrapper for fminunc that returns only cost"""
    J, grad = lrCostFunction(theta,X,y_one,lambda_)
    return J

## 1.4 One vs All classification

In [38]:
# Minimize using scipy fmin
import scipy.optimize as opt


def oneVsAll(num_labels, lambda_arg):
    """"trains LR classifier and return classifier params in matrix"""
    m = X.shape[0]
    n = X.shape[1]
    
    # This has to be returned
    all_theta = np.zeros((num_labels, n+1))
    
    # global var y_one
    
    lambda_ = lambda_arg
    
    
    # Add ones to X
    X = np.hstack((np.ones([m,1]), X))
    
    # Train classifier for each label
    for i in range(num_labels):
        y_one = (y == i)
        all_theta[i,:] = opt.fmin(decorated_lrCostFunction, all_theta[i,:], maxiter=400)

In [39]:
oneVsAll(9,0)

UnboundLocalError: local variable 'X' referenced before assignment